Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [416]:
#Create graph in this cell

# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.

beta = 0.001
learning_rate = 0.001
batch_size = 256

stepsPerEpoch = float(train_dataset.shape[0]) / batch_size

graph = tf.Graph()
with graph.as_default():
    
    def create_layer(data, weights, biases):
        return tf.matmul(data, weights) + biases
    
    def create_layer_relu(data, weights, biases):
        return tf.nn.relu(create_layer(data, weights, biases))
    
    def create_layer_relu_dropout(data, weights, biases):
        return tf.nn.dropout(create_layer_relu(data, weights, biases), 0.5)
    
    #create a list of weights matrices to use in the nn layers
    def create_weights_and_biases(layer_sizes):
        #it will be one smaller than layer_sizes list
        layer_weights = [{}] * (len(layer_sizes) - 1)
    
        for i in range(len(layer_sizes) - 1):
            wdict = {"weights":tf.Variable(tf.truncated_normal([layer_sizes[i], 
                                                                           layer_sizes[i + 1]])), 
                               "biases": tf.Variable(tf.zeros([layer_sizes[i + 1]])) } 
            layer_weights[i] = wdict
        return layer_weights
                
    
    def feed_forward(data, layer_weights):
        logit = data
        for i in range(len(layer_weights) - 1):       
            logit = create_layer_relu_dropout(logit, layer_weights[i]["weights"], layer_weights[i]["biases"])
            
        #the output layer doesn't use RELU
        last = len(layer_weights) - 1
        logit = create_layer(logit, layer_weights[last]["weights"], layer_weights[last]["biases"])
        return logit
    
    def feed_forward_validate(data, layer_weights):
        logit = data
        for i in range(len(layer_weights) - 1):
            #no dropout
            logit = create_layer_relu(logit, layer_weights[i]["weights"], layer_weights[i]["biases"])
            
        #the output layer doesn't use RELU
        last = len(layer_weights) - 1
        logit = create_layer(logit, layer_weights[last]["weights"], layer_weights[last]["biases"])
        return logit

    def create_layer_sizes():
        layer_sizes = [1024] * 4
        layer_sizes[0]= 28 * 28
        layer_sizes[-2] = 256
        layer_sizes[-1] = num_labels
        return layer_sizes
    
    def create_regularizers(weight_list):
        total = 0
        for w in weight_list:
            total += tf.nn.l2_loss(w["weights"])
            total += tf.nn.l2_loss(w["biases"])
        return total    
        

    # Input data.
    # Load the training, validation and test data into constants that are
    # attached to the graph.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    #create the layer sizes
    layer_sizes = create_layer_sizes()
    
    #create the weights
    weights_list = create_weights_and_biases(layer_sizes)

    #the neural network: feed forward
    logits = feed_forward(tf_train_dataset, weights_list)

    #regularizers, using l2 regularization
    regularizers = create_regularizers(weights_list)
    print(regularizers)

    #Softmax the result of the relu with expected values, create loss function    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + beta * regularizers 

    # Optimizer.
    global_step = tf.Variable(0)  # count the number of steps taken.
    #learning_rate = tf.train.exponential_decay(learning_rate, global_step, int(stepsPerEpoch) * 2, 0.96, staircase = True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

    # Predictions for the training, validation, and test data.
    # These are not part of training, but merely here so that we can report
    # accuracy figures as we train.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(feed_forward_validate(tf_valid_dataset, weights_list))
    test_prediction = tf.nn.softmax(feed_forward_validate(tf_test_dataset, weights_list))



Tensor("add_8:0", shape=(), dtype=float32)


In [415]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 11402.101562
Minibatch accuracy: 11.7%
Validation accuracy: 9.4%
Minibatch loss at step 500: 4536.011719
Minibatch accuracy: 55.9%
Validation accuracy: 75.5%
Minibatch loss at step 1000: 4155.370117
Minibatch accuracy: 62.5%
Validation accuracy: 75.8%


KeyboardInterrupt: 

---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---
